In [ ]:
import pyaudio
import sounddevice as sd

# Index des micros et fréquences supportées
p = pyaudio.PyAudio()

for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    print((i,dev['name'],dev['maxInputChannels']))

print(sd.query_devices())

samplerates = 16000, 32000, 44100, 48000, 96000, 128000
device = 1

supported_samplerates = []
for fs in samplerates:
    try:
        sd.check_output_settings(device=device, samplerate=fs)
    except Exception as e:
        print(fs, e)
    else:
        supported_samplerates.append(fs)
print(supported_samplerates)

In [ ]:
from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import subprocess
import srt
import json
import datetime
import pyaudio
import time
import ctypes
from multiprocessing import Process, Value, Array
import audioop

#manager = multiprocessing.Manager()

SetLogLevel(-1)
model_path = "../../asr/vosk/model"

if not os.path.exists(model_path):
	print (f"Please download the model from https://alphacephei.com/vosk/models and unpack as 'model' in {model_path}")
	exit (1)

model = Model(model_path)
rec = KaldiRecognizer(model, 16000)
rec.SetWords(True)

# ASR loop
def asr():
	p = pyaudio.PyAudio()
	stream = p.open(input_device_index=1, format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=8000)
	stream.start_stream()

	while True:
		data = stream.read(4000)
		#data = audioop.ratecv(data, 1, 1, 48000, 16000, None)[0]
		if len(data) == 0:
			break
		if rec.AcceptWaveform(data):
			result = rec.FinalResult()
			print(result)
			jres = json.loads(result)
			if "result" in jres:
				sentence = jres["text"]
				if "photo" in sentence:
					print("mot photo détecté !")
					print(":)")

asr()